# 5.- Test Environment Local

In [11]:
import os
import docker
from datetime import datetime

In [12]:
# Create a Docker client
client = docker.from_env()

In [13]:
# Docker image name
docker_image = 'satellite-object-detection:latest'

In [14]:
# Generar un directorio con fecha y hora
local_experiment_name = f"experiment_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
model_dir = f"/app/train/model_output/test_local_training/{local_experiment_name}"

In [15]:
data_local_path = os.path.abspath('../data')
train_local_path = os.path.abspath('../train')

In [16]:
# Corrected volume mounting with different local paths
volumes = {
    data_local_path: {'bind': '/app/data', 'mode': 'rw'},
    train_local_path: {'bind': '/app/train', 'mode': 'rw'},
}

In [17]:
# Define the command to execute the model training script
command = [
    'python', '/app/train/src/research/object_detection/model_main_tf2.py',
    '--pipeline_config_path', '/app/train/config_files/local_config_file.config',
    '--model_dir', model_dir,
    '--alsologtostderr',
    '--num_train_steps', '1',
    '--sample_1_of_n_eval_examples', '1'
]

In [ ]:
# Run the container and capture the output
try:
    # Run the container with bash, install libGL, then execute the command
    container = client.containers.run(
        docker_image,         # Docker image name
        command,
        volumes=volumes,      # Volumes to mount
        detach=True,          # Run in background (detach mode)
        stdout=True,          # Enable stdout capture
        stderr=True,          # Enable stderr capture
        tty=False             # Disable TTY, since we don’t need interactivity
    )

    print("Container is running...")

    # Collect and print logs in real-time
    for log in container.logs(stream=True):
        print(log.decode('utf-8'), end='')

    # Wait for the container to finish
    container.wait()

    print("Command executed successfully")

except docker.errors.DockerException as e:
    print(f"Error executing Docker command: {e}")
